In [ ]:
import os
import json
from google.colab import files
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

In [ ]:
print("Please upload your client_secret.json file")
CLIENT_SECRETS_FILE = files.upload()

print("Please upload your subscriptions.json file")
SUBSCRIPTIONS_FILE = files.upload()

print("Done!")

In [ ]:
CLIENT_SECRETS_FILE = '/content/client_secret.json'
SUBSCRIPTIONS_FILE = '/content/subscriptions.json'
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']

In [ ]:
def authenticate():
    flow = InstalledAppFlow.from_client_secrets_file(
        CLIENT_SECRETS_FILE, SCOPES, redirect_uri='urn:ietf:wg:oauth:2.0:oob')
    auth_url, _ = flow.authorization_url(prompt='consent')

    print('Please go to this URL: {}'.format(auth_url))
    code = input('Enter the authorization code: ')
    flow.fetch_token(code=code)

    credentials = flow.credentials
    return build('youtube', 'v3', credentials=credentials)

def read_subscriptions():
    with open(SUBSCRIPTIONS_FILE, 'r') as f:
        subscriptions_data = json.load(f)
    return subscriptions_data

def subscribe_to_channel(service, channel_id):
    try:
        subscription_response = service.subscriptions().insert(
            part='snippet',
            body={
                'snippet': {
                    'resourceId': {
                        'kind': 'youtube#channel',
                        'channelId': channel_id
                    }
                }
            }
        ).execute()
        print(f'Subscribed to channel ID {channel_id}')
    except HttpError as e:
        error = json.loads(e.content.decode('utf-8'))['error']
        if error['errors'][0]['reason'] == 'subscriptionDuplicate':
            print(f'Channel ID {channel_id} is already subscribed.')
        else:
            print(f'Error subscribing to channel ID {channel_id}: {error}')

In [ ]:
youtube_service = authenticate()

subscriptions_data = read_subscriptions()

for subscription in subscriptions_data['subscriptions']:
    channel_url = subscription['url']
    channel_id = channel_url.split('/')[-1]  # Extract channel ID from URL
    subscribe_to_channel(youtube_service, channel_id)